## LLM Zoomcamp 2024

Module 1: Introduction to LLMs and RAG

Homework

In [15]:
import requests
import os
import json
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

## Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

curl localhost:9200

What's the version.build_hash value?

In [1]:
!curl http://localhost:9200

{
  "name" : "ac31a2ce5169",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "wi3FaL8JSdyVpQqP9200Aw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data


Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

pip install elasticsearch

Which function do you use for adding your data to elastic?

insert
index
put
add

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
from elasticsearch import Elasticsearch


In [8]:
es_client = Elasticsearch("http://localhost:9200")

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questionss"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questionss'})

In [16]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:18<00:00, 50.91it/s]


In [17]:
doc

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp'}

## Q3. Searching
Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

94.05
84.05
74.05
64.05

Look at the _score field.

In [18]:
query = "How do I execute a command in a running docker container?"

In [19]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                }  
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

top_score = response["hits"]["hits"][0]["_score"]

print("Score for the top ranking result:", top_score)

Score for the top ranking result: 84.050095


In [23]:
response["hits"]["hits"][0]["_source"]

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I debug a docker container?',
 'course': 'machine-learning-zoomcamp'}

## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?

- How do I copy files from a different folder into docker container’s working directory?

- How do Lambda container images work?

- How can I annotate a graph?

In [24]:
def elastic_search(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    results_docs = []

    for hit in response["hits"]["hits"]:
        results_docs.append(hit["_source"])
 
    return results_docs

In [25]:
search_results = elastic_search(query)
third_question = search_results[2]["question"]
print("Third question:", third_question)

Third question: How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """
Q: {question}
A: {text}
""".strip()
Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
What's the length of the resulting prompt? (use the len function)

- 962
- 1462
- 1962
- 2462

In [42]:
def build_prompt(query, search_results):
    prompt_template = """
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    prompt_length = len(prompt)

    print("The length of the prompt is:", prompt_length)
    return prompt
    

In [43]:
prompt = build_prompt(query, search_results)
print(prompt)

The length of the prompt is: 1462
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

question: How do I copy files from my local machine to docker container?
answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_d

## Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

Let's calculate the number of tokens in our query:

Use the encode function. How many tokens does our prompt have?

- 122
- 222
- 322
- 422

Note: to decode back a token into a word, you can use the decode_single_token_bytes function:


In [31]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 17.5 MB/s eta 0:00:00m eta 0:00:01


In [38]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [39]:
prompt = build_prompt(query, search_results)

In [40]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-4o')

# Encoding the text and counting the tokens
token = encoding.encode(prompt)
num_tokens = len(token)

print("Number of tokens in the query:", num_tokens)

decoded= encoding.decode_single_token_bytes(63842)
print("Decoded example token:", decoded)

Number of tokens in the query: 322
Decoded example token: b"You're"
